In [ ]:
import pandas as pd
import geopandas as gpd
import movingpandas as mpd
import numpy as np
from datetime import timedelta, datetime
import time
from scipy.sparse import coo_matrix
from shapely.geometry import Point, LineString
import networkx as nx
import matplotlib.pyplot as plt
import folium
import warnings
import sys

warnings.filterwarnings('ignore')

print("Geopandas has version {}".format(gpd.__version__))
print("Movingpandas has version {}".format(mpd.__version__))

In [ ]:
# add paths for modules
sys.path.append('../src/models')
sys.path.append('../src/visualization')
sys.path.append('../src/features')
# import modules
import visualize
import geometry_utils
from maritime_traffic_network import MaritimeTrafficNetwork

In [ ]:
# read raw data from file
filename = '../data/processed/202204_points_stavanger_cleaned_meta_500k_dualSplit.parquet'
gdf = gpd.read_parquet(filename)

In [ ]:
# Transform to desired CRS
# 4326 for WGS 84 (global) // 32632 for UTM 32N (Norway)
crs = 32632  # Coordinate reference system
gdf.to_crs(crs, inplace=True)  # Transformation
gdf.head()

In [ ]:
# initialize maritime traffic network
network = MaritimeTrafficNetwork(gdf, crs)
network.get_trajectories_info()

In [ ]:
# compute Douglas Peucker significant points
network.calc_significant_points_DP(tolerance=10)

In [ ]:
# detect waypoints using spatial clustering
method = 'HDBSCAN'      # 'DBSCAN' , 'HDBSCAN', 'OPTICS'
metric = 'mahalanobis'  # 'euclidean', 'mahalanobis', 'haversine'
min_samples = 10
min_cluster_size = 10
eps = 0.02
V = np.diag([1, 1, 0.01, 0.01, 1e-5])  # mahalanobis distance parameter matrix V = np.diag([0.01, 0.01, 1e6, 1e6]) 
network.calc_waypoints_clustering(method=method, min_samples=min_samples, min_cluster_size=min_cluster_size,
                                  eps=eps, metric=metric, V=V)

In [ ]:
# build network
#network.make_graph_from_waypoints(min_passages=2)
start = time.time()
n_clusters = len(network.waypoints)
coord_dict = {}
wps = network.waypoints.copy()
wps.set_geometry('convex_hull', inplace=True)
# for each trajectory, find the distance to all waypoints
for mmsi in network.significant_points.mmsi.unique():
    # find all intersections and close passages of waypoints
    trajectory = network.significant_points_trajectory.get_trajectory(mmsi)
    trajectory_segments = trajectory.to_line_gdf()
    distances = trajectory.distance(wps.convex_hull)
    mask = distances<10
    close_wps = wps[mask]
    # find temporal order  of waypoint passage
    passages = []
    for i in range(0, len(trajectory_segments)):
        segment = trajectory_segments.iloc[i]
        # distance of each segment to the selection of close waypoints
        distance_to_line = segment['geometry'].distance(close_wps.convex_hull)  # distance between line segment and waypoint convex hull     
        distance_to_origin = segment['geometry'].boundary.geoms[0].distance(close_wps.geometry)  # distance between first point of segment and waypoint centroids (needed for sorting)
        close_wps['distance_to_line'] = distance_to_line.tolist()
        close_wps['distance_to_origin'] = distance_to_origin.tolist()
        # angle between line segment and mean traffic direction in each waypoint
        close_wps['angle'] = np.abs(geometry_utils.compass_mean(close_wps['cog_before'], close_wps['cog_after']) - segment['direction'])  
        # the line segment is associated with the waypoint, when its distance and angle is less than a threshold
        passed_wps = close_wps[(close_wps['distance_to_line']<10) & (close_wps['angle']<45)]
        passed_wps.sort_values(by='distance_to_origin', inplace=True)
        passages.extend(passed_wps['clusterID'].tolist())
    
    # create edges between subsequent passed waypoints
    if len(passages) > 1:  # subset needs to contain at least 2 waypoints
        for i in range(0, len(passages)-1):
            row = passages[i]
            col = passages[i+1]
            if row != col:  # no self loops
                if ((row==184) & (col==181)):
                    print(mmsi)
                if (row, col) in coord_dict:
                    coord_dict[(row, col)] += 1  # increase the edge weight for each passage
                else:
                    coord_dict[(row, col)] = 1  # create edge if it does not exist yet

In [ ]:
# store adjacency matrix as sparse matrix in COO format
row_indices, col_indices = zip(*coord_dict.keys())
values = list(coord_dict.values())
A = coo_matrix((values, (row_indices, col_indices)), shape=(n_clusters, n_clusters))

# Construct a GeoDataFrame from graph edges
waypoints = network.waypoints.copy()
waypoints.set_geometry('geometry', inplace=True, crs=network.crs)
waypoint_connections = pd.DataFrame(columns=['from', 'to', 'geometry', 'direction', 'passages'])
for orig, dest, weight in zip(A.row, A.col, A.data):
    # add linestring as edge
    p1 = waypoints[waypoints.clusterID == orig].geometry
    p2 = waypoints[waypoints.clusterID == dest].geometry
    edge = LineString([(p1.x, p1.y), (p2.x, p2.y)])
    # compute the orientation fo the edge (COG)
    p1 = Point(waypoints[waypoints.clusterID == orig].lon, waypoints[waypoints.clusterID == orig].lat)
    p2 = Point(waypoints[waypoints.clusterID == dest].lon, waypoints[waypoints.clusterID == dest].lat)
    direction = geometry_utils.calculate_initial_compass_bearing(p1, p2)
    line = pd.DataFrame([[orig, dest, edge, direction, weight]], 
                        columns=['from', 'to', 'geometry', 'direction', 'passages'])
    waypoint_connections = pd.concat([waypoint_connections, line])
waypoint_connections = gpd.GeoDataFrame(waypoint_connections, geometry='geometry', crs=network.crs)
end = time.time()
print(f'Time elapsed: {(end-start)/60:.2f} minutes')

In [ ]:
G = nx.from_scipy_sparse_array(A, create_using=nx.DiGraph)

# add node features
for i in range(0, len(waypoints)):
    node_id = waypoints.clusterID.iloc[i]
    G.nodes[node_id]['n_members'] = waypoints.n_members.iloc[i]
    G.nodes[node_id]['position'] = (waypoints.lon.iloc[i], waypoints.lat.iloc[i])  # !changed lat-lon to lon-lat for plotting
    G.nodes[node_id]['speed'] = waypoints.speed.iloc[i]
    G.nodes[node_id]['cog_before'] = waypoints.cog_before.iloc[i]
    G.nodes[node_id]['cog_after'] = waypoints.cog_after.iloc[i]

In [ ]:
mmsi = ['219347000_4_2022-04-03 01:38:48', '219348000_0_2022-04-02 01:42:01', '305981000_0_2022-04-02 11:03:05']
#trajectory = network.trajectories.get_trajectory(mmsi)
#DP_trajectory = network.significant_points_trajectory.get_trajectory(mmsi)
DP_points = network.significant_points[network.significant_points.mmsi.isin(mmsi)]
DP_trajectories = network.significant_points_trajectory.to_traj_gdf()

In [ ]:
#cluster_list=[357, 358, 389, 478, 389, 477, 478, 477, 478, 477, 510, 511, 373, 430, 427, 429, 430, 461, 462, 459, 460, 462, 474, 473, 474, 475, 473, 483, 494, 494, 494, 495, 597, 597, 597, 598, 
#              599, 600, 598, 599, 600, 152, 153, 598, 599, 600, 604, 605, 152, 153, 604, 605, 153, 504, 605]
#wpts = network.waypoints.copy()
#wpts.set_geometry('convex_hull', inplace=True, crs=network.crs)

In [ ]:
map = network.map_waypoints()
map = waypoint_connections[waypoint_connections.passages>1].explore(m=map)
#map = network.map_graph(pruned=True)
#map = visualize.map_accurate_and_simplified_trajectory(trajectory, DP_trajectory, map=map)
map = DP_trajectories[DP_trajectories.mmsi.isin(mmsi)][['geometry', 'mmsi']].explore(m=map, color='black')
map = DP_points[DP_points.clusterID >= 0][['geometry', 'clusterID', 'mmsi']].explore(m=map, color='pink')
#map = wpts[wpts.clusterID.isin(cluster_list)][['convex_hull', 'clusterID']].explore(m=map, name='passed clusters', legend=False,
#                                              style_kwds={'color':'yellow', 'fillColor':'yellow', 'fillOpacity':0.2})
folium.LayerControl().add_to(map)
map